In [52]:
import os
import cv2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

In [2]:
os.environ['KAGGLE_USERNAME'] = "lorddexter"
os.environ['KAGGLE_KEY'] = "bf0fec1866a689351f73244d4dd09a99"
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

100% 673M/675M [00:03<00:00, 228MB/s]
100% 675M/675M [00:03<00:00, 198MB/s]


In [3]:
!unzip cell-images-for-detecting-malaria.zip

Streaming output truncated to the last 5000 lines.
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_118.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_126.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_134.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_141.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_168.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_175.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_183.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_221.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_222.png  
  inflating: cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_87.png  
  inflating: cell_im

In [4]:
base_path = '/content/cell_images/cell_images'

In [5]:
def load_and_preprocess_images(base_path):

  Image_files = []
  Labels = []
  Dataset = []
  
  for label in os.listdir(base_path):

    # append labels from sub-directory names
    Labels.append(label)
    
    # creating image path
    images_path = os.path.join(base_path, label)
    
    for image_name in os.listdir(images_path):

      image_paths = os.path.join(images_path, image_name)
      
      # read the images
      img = cv2.imread(image_paths)

      try:
        
        # resizing the images
        img = cv2.resize(img, (100, 100), cv2.INTER_CUBIC)

        # make copy of resized image for edge detection
        img_copy = img.copy()
    
        # convert the images to grayscale for edge detection
        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)   
        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)

        # preprocessing using canny edge detector to make image edges sharper and image smoother
        img_copy = cv2.GaussianBlur(img, (7,7), 0) 
        edges = cv2.Canny(img_copy, threshold1 = 80, threshold2 = 160)
        edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        
        # superimposing image with detected edges
        final_img = cv2.addWeighted(img, 0.5, edges, 0.5, 0)
        
        Dataset.append([final_img, np.array(label)])
      except:
        continue

  return Dataset

In [6]:
Dataset = load_and_preprocess_images(base_path)

In [7]:
Data = pd.DataFrame(Dataset, columns = ['Images', 'Labels'])

In [8]:
Data.head()

,Images,Labels
0,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Parasitized
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Parasitized
2,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Parasitized
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Parasitized
4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Parasitized


In [9]:
def Labels_to_numeric(x):
        if x == 'Parasitized':
          return 1
        if x == 'Uninfected':
          return 0

Data['Labels'] = Data['Labels'].apply(Labels_to_numeric)
print(Data.head())

                                              Images  Labels
0  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       1
1  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       1
2  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       1
3  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       1
4  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...       1


In [14]:
x = np.array([i for i in Data['Images']]).reshape(-1, 100, 100,3)
y = to_categorical([i for i in Data['Labels']])

In [15]:
X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, shuffle = True, random_state = 2)

In [16]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(22046, 100, 100, 3)
(5512, 100, 100, 3)
(22046, 2)
(5512, 2)


In [18]:
# defining evaluation metrics

precision = Precision()
recall = Recall(thresholds = 0.2)

### Trying out MobileNet architecture

In [19]:
mobilenet = MobileNetV2(input_shape = (100, 100, 3), alpha = 1.0, include_top = False)
mobilenet.trainable = False

9420800/9406464 [==============================] - 0s 0us/step


In [20]:
first = Flatten()(mobilenet.output)
conv1 = Dense(units = 600, activation = 'relu')(first)
dropout = Dropout(0.5)(conv1)
last = Dense(units = 2, activation = 'sigmoid')(dropout)
mobilenet_model = Model(inputs = mobilenet.input, outputs = last)

In [22]:
mobilenet_adam = Adam(learning_rate = 0.00000001)
mobilenet_sgd = SGD(learning_rate = 0.00000001, momentum = 0.9, nesterov = True)


mobilenet_model.compile(optimizer = mobilenet_sgd, loss = 'categorical_crossentropy', metrics = ['accuracy', precision, recall])

In [23]:
class myCallback(tensorflow.keras.callbacks.Callback):

  def on_epoch_end(self, epochs, logs = {}):
    if (logs.get('accuracy') > 0.9):
      self.model.stop_training = True

myCallback = myCallback()

mobilenet_rlop = ReduceLROnPlateau(monitor = 'accuracy', patience = 5)

In [24]:
mobilenet_model.fit(x = X_train, y = Y_train, validation_data=[X_val, Y_val], callbacks = [mobilenet_rlop, myCallback], epochs = 30)

Epoch 1/30
689/689 [==============================] - 43s 44ms/step - loss: 3.0094 - accuracy: 0.5044 - precision_1: 0.5161 - recall: 0.4685 - val_loss: 2.4251 - val_accuracy: 0.4942 - val_precision_1: 0.5407 - val_recall: 0.4679 - lr: 1.0000e-08
Epoch 2/30
689/689 [==============================] - 27s 40ms/step - loss: 2.3726 - accuracy: 0.5048 - precision_1: 0.5102 - recall: 0.4553 - val_loss: 1.7405 - val_accuracy: 0.4998 - val_precision_1: 0.5655 - val_recall: 0.4465 - lr: 1.0000e-08
Epoch 3/30
689/689 [==============================] - 28s 40ms/step - loss: 1.9592 - accuracy: 0.4989 - precision_1: 0.5134 - recall: 0.4495 - val_loss: 1.3323 - val_accuracy: 0.5022 - val_precision_1: 0.5674 - val_recall: 0.4320 - lr: 1.0000e-08
Epoch 4/30
689/689 [==============================] - 27s 40ms/step - loss: 1.7522 - accuracy: 0.5029 - precision_1: 0.5124 - recall: 0.4462 - val_loss: 1.1178 - val_accuracy: 0.5076 - val_precision_1: 0.5499 - val_recall: 0.4169 - lr: 1.0000e-08
Epoch 5/30
6

In [33]:
mobilenet_model.save(filepath = 'mobilenet_model', save_format = 'h5', include_optimizer = True)

In [35]:
mobilenet_model.save(filepath = 'mobilenet_model_tf', save_format = 'tf', include_optimizer = True)

INFO:tensorflow:Assets written to: mobilenet_model_tf/assets


### Trying out VGG19 model

In [25]:
vgg = VGG19(include_top=False, weights = 'imagenet', input_shape = (100, 100, 3))
vgg.trainable = False

80150528/80134624 [==============================] - 1s 0us/step


In [26]:
x = Flatten()(vgg.output)
dense1 = Dense(units=128,activation='relu')(x)
dense2 = Dense(units=128, activation='relu')(dense1)
output = Dense(units=2, activation='sigmoid')(dense2)
vgg_model = Model(inputs = vgg.input, outputs = output)

In [27]:
vgg_adam = Adam(learning_rate = 0.0000001)
vgg_sgd = SGD(learning_rate = 0.000001, momentum = 0.8, nesterov = True)

vgg_rlop = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 10, min_lr = 0.00000000001, verbose = 1, factor = 0.2)

vgg_model.compile(optimizer = vgg_sgd, metrics = ['accuracy', precision, recall], loss = 'categorical_crossentropy')

In [28]:
vgg_model.fit(x = X_train, y = Y_train, validation_data = [X_val, Y_val], callbacks = [vgg_rlop, myCallback], epochs = 30)

Epoch 1/30
689/689 [==============================] - 88s 124ms/step - loss: 2.1026 - accuracy: 0.6291 - precision_1: 0.6005 - recall: 0.6638 - val_loss: 1.0800 - val_accuracy: 0.7600 - val_precision_1: 0.6818 - val_recall: 0.8155 - lr: 1.0000e-06
Epoch 2/30
689/689 [==============================] - 83s 120ms/step - loss: 0.9306 - accuracy: 0.7845 - precision_1: 0.6879 - recall: 0.8502 - val_loss: 0.7619 - val_accuracy: 0.8117 - val_precision_1: 0.7004 - val_recall: 0.8770 - lr: 1.0000e-06
Epoch 3/30
689/689 [==============================] - 83s 121ms/step - loss: 0.7225 - accuracy: 0.8221 - precision_1: 0.6949 - recall: 0.8904 - val_loss: 0.6275 - val_accuracy: 0.8394 - val_precision_1: 0.6976 - val_recall: 0.9038 - lr: 1.0000e-06
Epoch 4/30
689/689 [==============================] - 83s 120ms/step - loss: 0.6171 - accuracy: 0.8395 - precision_1: 0.6905 - recall: 0.9094 - val_loss: 0.5486 - val_accuracy: 0.8549 - val_precision_1: 0.6911 - val_recall: 0.9191 - lr: 1.0000e-06
Epoch 5/

In [32]:
vgg_model.save(filepath = 'vgg_model', include_optimizer = True, save_format = 'h5')

In [34]:
vgg_model.save(filepath = 'vgg_model_tf', include_optimizer = True, save_format = 'tf')

INFO:tensorflow:Assets written to: vgg_model_tf/assets


### Trying Resnet-152 model

In [36]:
resnet = ResNet152(include_top = False, weights = 'imagenet', input_shape = (100, 100, 3))
resnet.trainable = False

234708992/234698864 [==============================] - 3s 0us/step


In [37]:
x = Flatten()(resnet.output)
dense1 = Dense(units=128,activation='relu')(x)
output = Dense(units=2, activation='sigmoid')(dense1)
resnet_model = Model(inputs = resnet.input, outputs = output)

In [39]:
resnet_adam = Adam(learning_rate = 0.0000001)
resnet_sgd = SGD(learning_rate = 0.000001, momentum = 0.8, nesterov = True)

resnet_model.compile(optimizer = resnet_adam, loss = 'binary_crossentropy', metrics = ['accuracy', precision, recall])

In [40]:
resnet_model.fit(x = X_train, y = Y_train, validation_data = [X_val, Y_val], epochs = 30, callbacks = [myCallback])

Epoch 1/30
689/689 [==============================] - 164s 219ms/step - loss: 0.7520 - accuracy: 0.7249 - precision_1: 0.7003 - recall: 0.7941 - val_loss: 0.5756 - val_accuracy: 0.7792 - val_precision_1: 0.7522 - val_recall: 0.8855
Epoch 2/30
689/689 [==============================] - 152s 221ms/step - loss: 0.5099 - accuracy: 0.8054 - precision_1: 0.7755 - recall: 0.9150 - val_loss: 0.4537 - val_accuracy: 0.8309 - val_precision_1: 0.8043 - val_recall: 0.9269
Epoch 3/30
689/689 [==============================] - 148s 215ms/step - loss: 0.4190 - accuracy: 0.8428 - precision_1: 0.8200 - recall: 0.9399 - val_loss: 0.3842 - val_accuracy: 0.8579 - val_precision_1: 0.8369 - val_recall: 0.9412
Epoch 4/30
689/689 [==============================] - 147s 214ms/step - loss: 0.3629 - accuracy: 0.8660 - precision_1: 0.8458 - recall: 0.9496 - val_loss: 0.3388 - val_accuracy: 0.8777 - val_precision_1: 0.8581 - val_recall: 0.9497
Epoch 5/30
689/689 [==============================] - 142s 206ms/step - 

In [41]:
resnet_model.save(filepath = 'resnet_model', save_format = 'h5', include_optimizer = True)

In [43]:
resnet_model.save(filepath = 'resnet_model_tf', save_format = 'tf', include_optimizer = True)

INFO:tensorflow:Assets written to: resnet_model_tf/assets


### Trying InceptionV3

In [45]:
inception = InceptionV3(include_top = False, input_shape = (100, 100, 3), weights = 'imagenet')
inception.trainable = False

87924736/87910968 [==============================] - 1s 0us/step


In [47]:
x = Flatten()(inception.output)
dense1 = Dense(units = 1012, activation = 'relu')(x)
dense2 = Dense(units = 504, activation = 'relu')(dense1)
dense3 = Dense(units=128,activation='relu')(dense2)
output = Dense(units=2, activation='sigmoid')(dense3)
inception_model = Model(inputs = inception.input, outputs = output)

In [48]:
inception_adam = Adam(learning_rate = 0.0000001)
inception_sgd = SGD(learning_rate = 0.000001, momentum = 0.8, nesterov = True)

inception_model.compile(optimizer = inception_adam, loss = 'binary_crossentropy', metrics = ['accuracy', precision, recall])

In [49]:
inception_model.fit(x = X_train, y = Y_train, validation_data = [X_val, Y_val], epochs = 30, callbacks = [myCallback])

Epoch 1/30
689/689 [==============================] - 61s 77ms/step - loss: 6.2546 - accuracy: 0.4931 - precision_1: 0.5474 - recall: 0.9643 - val_loss: 3.0851 - val_accuracy: 0.5194 - val_precision_1: 0.5001 - val_recall: 0.8821
Epoch 2/30
689/689 [==============================] - 51s 74ms/step - loss: 2.2069 - accuracy: 0.5352 - precision_1: 0.5190 - recall: 0.7482 - val_loss: 1.7498 - val_accuracy: 0.5796 - val_precision_1: 0.5530 - val_recall: 0.7065
Epoch 3/30
689/689 [==============================] - 51s 74ms/step - loss: 1.6063 - accuracy: 0.5965 - precision_1: 0.5737 - recall: 0.7086 - val_loss: 1.4007 - val_accuracy: 0.6377 - val_precision_1: 0.6037 - val_recall: 0.7471
Epoch 4/30
689/689 [==============================] - 51s 74ms/step - loss: 1.3128 - accuracy: 0.6502 - precision_1: 0.6204 - recall: 0.7505 - val_loss: 1.1798 - val_accuracy: 0.6825 - val_precision_1: 0.6471 - val_recall: 0.7747
Epoch 5/30
689/689 [==============================] - 49s 72ms/step - loss: 1.12

In [50]:
inception_model.save(filepath = 'inception_model', save_format = 'h5', include_optimizer = True)

In [51]:
inception_model.save(filepath = 'inception_model_tf', save_format = 'tf', include_optimizer = True)

INFO:tensorflow:Assets written to: inception_model_tf/assets


### Trying InceptionResNetV2

In [53]:
inception_resnet = InceptionResNetV2(include_top = False, weights = 'imagenet', input_shape = (100, 100, 3))
inception_resnet.trainable = False

219070464/219055592 [==============================] - 3s 0us/step


In [56]:
x = Flatten()(inception_resnet.output)
dense1 = Dense(units = 504, activation = 'relu')(x)
dense2 = Dense(units=128,activation='relu')(dense1)
output = Dense(units=2, activation='sigmoid')(dense2)
inception_resnet_model = Model(inputs = inception_resnet.input, outputs = output)

In [57]:
inception_resnet_adam = Adam(learning_rate = 0.0000001)
inception_resnet_sgd = SGD(learning_rate = 0.000001, momentum = 0.8, nesterov = True)

inception_resnet_model.compile(optimizer = inception_resnet_adam, loss = 'binary_crossentropy', metrics = ['accuracy', precision, recall])

In [58]:
inception_resnet_model.fit(x = X_train, y = Y_train, validation_data = [X_val, Y_val], epochs = 30, callbacks = [myCallback])

Epoch 1/30
689/689 [==============================] - 123s 157ms/step - loss: 15.3635 - accuracy: 0.4986 - precision_1: 0.5510 - recall: 0.6654 - val_loss: 5.9052 - val_accuracy: 0.5011 - val_precision_1: 0.4969 - val_recall: 0.8124
Epoch 2/30
689/689 [==============================] - 101s 147ms/step - loss: 1.9985 - accuracy: 0.4695 - precision_1: 0.4923 - recall: 0.7979 - val_loss: 1.1866 - val_accuracy: 0.4902 - val_precision_1: 0.5124 - val_recall: 0.7986
Epoch 3/30
689/689 [==============================] - 103s 149ms/step - loss: 1.0903 - accuracy: 0.5033 - precision_1: 0.5222 - recall: 0.8376 - val_loss: 1.0407 - val_accuracy: 0.5060 - val_precision_1: 0.5290 - val_recall: 0.8398
Epoch 4/30
689/689 [==============================] - 101s 147ms/step - loss: 0.9869 - accuracy: 0.5262 - precision_1: 0.5388 - recall: 0.8672 - val_loss: 0.9756 - val_accuracy: 0.5145 - val_precision_1: 0.5340 - val_recall: 0.8612
Epoch 5/30
689/689 [==============================] - 102s 149ms/step -

In [59]:
inception_resnet_model.save(filepath = 'inception_resnet_model', save_format = 'h5', include_optimizer = True)

In [60]:
inception_resnet_model.save(filepath = 'inception_resnet_model_tf', save_format = 'tf', include_optimizer = True)

INFO:tensorflow:Assets written to: inception_resnet_model_tf/assets
